### Importing Libraries

In [ ]:
import gc #garbage collector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import json
import sys

sys.path.append("..") # allow module discovery in parent directory

import warnings
warnings.filterwarnings('ignore')

from helper import utility
from helper.utility import NACellFillers

import importlib
importlib.reload(utility)

DATA_LOC: str = "../data"


### Importing Zillow Datasets

In [ ]:
# Raw data set provided
prop_2016 = utility.load_data(f'{DATA_LOC}/properties_2016.csv')
prop_2017 = utility.load_data(f'{DATA_LOC}/properties_2017.csv')
train_2016 = utility.load_data(f'{DATA_LOC}/train_2016_v2.csv' , ['transactiondate'])
train_2017 = utility.load_data(f'{DATA_LOC}/train_2017.csv', ['transactiondate'])

In [ ]:
# Combining the prop dataset with its corresponding train datasets on their parcelid
# left join is used so that all properties without logerror will be ignored
training_2016 = utility.merge_data(train_2016, prop_2016, 'parcelid')
training_2017 = utility.merge_data(train_2017, prop_2017, 'parcelid')



# Data across the 2 years are combined into one data frame for processing at later stages
training_all = pd.concat([training_2016, training_2017] , ignore_index=True)
properties_all = pd.concat([prop_2016, prop_2017], ignore_index=True)


training_all
#properties_all.shape


In [ ]:
# Check and Drop any duplicates in the training dataset
# Duplicates are those which have the same parcelid and transactiondate

training_all.shape
utility.check_duplicates(training_all)
training_all = utility.drop_dups(training_all)

In [ ]:
# We can see that there are no duplicates in the dataset so far
training_all.shape

### Looking at the Target Variable - logerror

In [ ]:
target_y = training_all['logerror']


target_y.hist(bins=50, figsize=(8,4))
plt.show()

In [ ]:
target_y.describe()

In [ ]:
# Drop outliers that are more than 2.5 std away from mean
upper_threshold = target_y.mean() + (2.5*target_y.std())
lower_threshold = target_y.mean() - (2.5*target_y.std())


# Remove data that have their target y value as outliers
training_all = training_all[training_all['logerror'] < upper_threshold]
training_all = training_all[training_all['logerror'] > lower_threshold]
training_all.shape


### Data Preprocessing & Feature Engineering

In [ ]:
# Adding Feature to the dataset
# Add Day, Month, Year and which quarter the transaction was done
training_all = utility.add_dmy_feature(training_all)
training_all

In [ ]:
utility.print_percent_missing(training_all)

In [ ]:
# Drop all columns that have missing threashold greater than 95%
MISSING_THRESHOLD = 0.97
col_to_drop = utility.get_col_to_drop_missing(training_all, 0.95)
col_to_drop += utility.get_col_to_drop_non_unique(training_all)

# Other columns to exlude to prepare for training dataset
exclude_list = ["parcelid" , "logerror" , 'propertyzoningdesc']

remaining_col = []
for col in training_all.columns:
    if col not in col_to_drop and col not in exclude_list:
        remaining_col.append(col)
        print(col)



In [ ]:
# Dealing with Categorical Values
# Convert categorical values to 'category' type for some columns

categorylist = ['airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid',
                'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid' ,
                'propertylandusetypeid', 'regionidcity', 'regionidcounty',
                'regionidneighborhood', 'storytypeid', 'typeconstructiontypeid']

for col in training_all.columns:
    if col in categorylist:
        utility.float_to_categorical(training_all, col)


In [ ]:
# Convert float64 values to float32 values
for col in training_all.columns:
    if training_all[col].dtype.name == 'float64':
        training_all[col] = training_all[col].astype('float32')

In [ ]:
# populate NA values
training_all.info()

In [ ]:
# get cols that contain NA cells
na_df = training_all.isna()
na_cols: pd.Series = na_df.any()
cols_with_na: list = na_cols[na_cols == True].index.tolist()

df_with_na = training_all[[*cols_with_na]]
df_with_na.info()

total = len(training_all)

#
if True:
    for col in df_with_na.columns:
        ser = df_with_na[col]
        non_null = ser.notna().sum()

        try:
            plt.title(f"{col}, non-null: {non_null}/{total}")
            sns.violinplot(ser)
            plt.show()

        except Exception as e:
            plt.cla()
            plt.clf()
            print(f"unable to plot for {col}: {e}, trying catplot")
            res = ser.value_counts().sort_values(ascending=False)

            plt.figure(figsize=(10, len(res)/5))
            plt.title(f"{col}, non-null: {non_null}/{total}")
            sns.barplot(x=res.tolist(), y=res.index)
            plt.show()


In [ ]:
# drop even more cols
# some cols have very few entries that are filled
# AND they do not contribute meaningful information
# these are dropped here

res = training_all.drop(
    columns=[
        # "decktypeid", # insufficient values; all the same
        "finishedsquarefeet15", # too little values
        "finishedsquarefeet13", # too little values
        "finishedsquarefeet6",
        "finishedsquarefeet50", # same as above

        "propertycountylandusecode",    # weird distribution of categories
        "propertyzoningdesc",           # same as above

        "assessmentyear",
        "landtaxvaluedollarcnt",
        "taxamount",
        "taxdelinquencyflag",
        "taxdelinquencyyear",
        "censustractandblock",
    ]
)

training_all = res
res.info()

In [ ]:
# constructing a Series to fill main dataframe

# create na_cols again from dropped dataframe
na_cols = training_all.isna().any()
cols_with_na: list = na_cols[na_cols == True].index.tolist()


fill_series: pd.Series = pd.Series(index=cols_with_na)

fill_series["basementsqft"] = 0 # no basement
fill_series["bathroomcnt"] = NACellFillers.mode(training_all["bathroomcnt"])
fill_series["bedroomcnt"] = NACellFillers.mode(training_all["bedroomcnt"])
fill_series["calculatedbathnbr"] = NACellFillers.mode(training_all["calculatedbathnbr"])
fill_series["decktypeid"] = NACellFillers.mode(training_all["decktypeid"])
fill_series["finishedfloor1squarefeet"] = NACellFillers.median(training_all["finishedfloor1squarefeet"])
fill_series["calculatedfinishedsquarefeet"] = NACellFillers.median(training_all["calculatedfinishedsquarefeet"])
fill_series["finishedsquarefeet12"] = NACellFillers.median(training_all["finishedsquarefeet12"])
# fill_series["finishedsquarefeet13"] = 0 # perimeter of living area (???)
# fill_series["finishedsquarefeet15"] = NACellFillers.median(training_all["finishedsquarefeet15"])
# fill_series["finishedsquarefeet50"] = NACellFillers.median(training_all["finishedsquarefeet50"])
fill_series["finishedsquarefeet6"] = 0 # base unfinished and finished area
fill_series["fireplacecnt"] = 0 # assume the rest do not have fireplaces
fill_series["fullbathcnt"] = NACellFillers.mode(training_all["fullbathcnt"])
fill_series["garagecarcnt"] = 0 # assume the rest do not have garages
fill_series["garagetotalsqft"] = 0 # same as above
fill_series["hashottuborspa"] = 0 # assume the rest do not have hot tubs/spa
fill_series["latitude"] = 0     # will be derived from their zip codes
fill_series["longitude"] = 0    # will be derived from their zip codes
fill_series["lotsizesquarefeet"] = NACellFillers.median(training_all["lotsizesquarefeet"])
fill_series["poolcnt"] = 0          # assume the rest do not have pools
fill_series["poolsizesum"] = 0      # same as above
fill_series["pooltypeid10"] = 0     # same
fill_series["pooltypeid2"] = 0      # same
fill_series["pooltypeid7"] = 0      # same
# fill_series["propertycountylandusecode"] = 0    # text cat, need to encode or something
# fill_series["propertyzoningdesc"] = 0           # text cat, need to encode or something
fill_series["rawcensustractandblock"] = NACellFillers.median(training_all["rawcensustractandblock"])
fill_series["regionidzip"] = 0  # will be derived from coordinates
fill_series["roomcnt"] = NACellFillers.mode(training_all["roomcnt"])
fill_series["threequarterbathnbr"] = 0 # assume home does not have 3/4 bathroom
fill_series["unitcnt"] = NACellFillers.mode(training_all["unitcnt"])
fill_series["yardbuildingsqft17"] = 0 # assume the rest do not have patios
fill_series["yardbuildingsqft26"] = 0 # assume the rest do not have storage shed/buiding
fill_series["yearbuilt"] = NACellFillers.median(training_all["yearbuilt"])
fill_series["numberofstories"] = NACellFillers.mode(training_all["numberofstories"])
fill_series["fireplaceflag"] = 0 # assume the rest do not have fireplaces
fill_series["structuretaxvaluedollarcnt"] = 0 # very little samples
fill_series["taxvaluedollarcnt"] = NACellFillers.median(training_all["taxvaluedollarcnt"])
# fill_series["assessmentyear"] = 0 # these should have been filled up already
# fill_series["landtaxvaluedollarcnt"] = NACellFillers.median(training_all["landtaxvaluedollarcnt"])
# fill_series["taxamount"] = NACellFillers.median(training_all["taxamount"])
# fill_series["taxdelinquencyflag"] = 0   # related to the two below
# fill_series["taxdelinquencyyear"] = 0   #
# fill_series["censustractandblock"] = 0  #

# save fill data
fill_series.to_json(f"{DATA_LOC}/na_fill_data.json", indent=4)

print("NA cell fill data:", end="\n\n")
print(fill_series)

In [ ]:
# latlong data
na_zip = training_all[training_all["regionidzip"].isna()]
na_lat = training_all[training_all["latitude"].isna()]
na_lon = training_all[training_all["longitude"].isna()]

s_zip = set(na_zip.index.tolist())
s_lat = set(na_lat.index.tolist())
s_lon = set(na_lon.index.tolist())

# indices with no lat, lon or zip code
i_all_na = s_zip.intersection(s_lon)

print(len(i_all_na))

In [ ]:
# drop indices
res: pd.DataFrame = training_all.drop(index=i_all_na)

# replace NA
res = res.fillna(fill_series)

res.reset_index(drop=True, inplace=True)

res.info()

In [ ]:
# some cells contain "True" instead of 1
for col in res.columns:
    continue

# gather the cols with not-co-clean data (the col dtype shows as 'object')
unclean_cols: list = []
for idx, typ in enumerate(res.dtypes):
    if typ == object:
        print("ahhh objet")
        unclean_cols.append(res.dtypes.index[idx])

print(unclean_cols)

In [ ]:
col = res['hashottuborspa']

def strbool_to_float(col: pd.Series):
    '''Maps a bool string to float with the following rule:
    - "True" => 1.0f
    - "False" => 0.0f
    - _ => 0.0f
    '''
    for idx in range(len(col)):
        if not isinstance(col[idx], float) \
            and isinstance(col[idx], str):

                if col[idx] == "True":
                    col[idx] = float(1.0)
                else:
                    col[idx] = 0.0

        else:
            col[idx] = 0.0

    return

for col_name in unclean_cols:
    strbool_to_float(res[col_name])


In [ ]:
# save
res.to_csv(f"{DATA_LOC}/train.csv", index=False)